# "Smooth" statistics & smooth line plot

Statistics "smooth" are calculated on the sample of two continuous variables (i.e., sample of points or lines).
It interpolates data points to create a smoother curve.

This notebook uses definitions from [DataFrame](https://kotlin.github.io/dataframe/overview.html).

## Usage

The "Smooth" statistic proves beneficial in scenarios with over-plotting or noise,
simplifying the process of identifying inherent trends and patterns.
It can also be used to make a more pretty line with a small number of points.

## Arguments

* Input (mandatory):
    - `x` — numeric sample of input points `x` coordinates
    - `y` — numeric sample of input points `y` coordinates
* Parameters (optional):
    - `method: SmoothMethod` — smoothing model:
        - `SmoothMethod.Linear(confidenceLevel: Double)` — linear model
        - `SmoothMethod.Polynomial(degree: Int, confidenceLevel: Double)` — polynomial model
        - `SmoothMethod.LOESS(span: Double, loessCriticalSize: Int, samplingSeed: Long, confidenceLevel: Double)` —
          Local Polynomial Regression model
    - `smootherPointCount: Int` — number of sampled points

### Generalized signature

The specific signature depends on the function,
but all functions related to "smooth" statistic (which will be discussed further below —
different variations of `statSmooth()`, `smoothLine()`) have approximately the same signature with the arguments above:

```
statSmoothArgs := 
   x, 
   y,
   method: SmoothMethod = SmoothMethod.LOESS(),
   smootherPointCount: Int = 100
```

The possible types of `x` and `y` depend on where a certain function is used.
They can be simply `Iterable` (`List`, `Set`, etc.) or a reference to a column in a `DataFrame`
(`String`, `ColumnAccessor`) or the `DataColumn` itself.

## Output statistics

| name      | type   | description                                          |
|-----------|--------|------------------------------------------------------|
| Stat.x    | Double | `x` coordinate                                       |
| Stat.y    | Double | `y` coordinate                                       |
| Stat.yMin | Double | Lower point-wise confidence interval around the mean |
| Stat.yMax | Double | Upper point-wise confidence interval around the mean |
| Stat.se   | Double | Standard error                                       |

## StatSmooth plots

In [1]:
%useLatestDescriptors
%use dataframe
%use kandy

import org.apache.commons.math3.distribution.NormalDistribution
import org.apache.commons.math3.distribution.UniformRealDistribution
import kotlin.random.Random

In [2]:
// To generate the data, we use a standard java math library
// https://commons.apache.org/proper/commons-math/

// Generate line with formula
val xs = (-100..100).map { it / 50.0 }
val lineFormula = { x: Double -> 2.0 / (x * x + 0.5) }
// Generate noises from normal distribution
val noises = NormalDistribution(0.0, 0.1).sample(xs.size).toList()
val ys = xs.zip(noises).map { lineFormula(it.first) + it.second }
// And drop 2/3 points
val random = Random(42)
val (newXs, newYs) = xs.zip(ys).shuffled(random).take(xs.size * 1 / 3).sortedBy { it.first }.unzip()

In [3]:
// Gather them into the DataFrame
val df = dataFrameOf(
    "speed" to newXs,
    "efficiency" to newYs
)
df.head(5)

speed,efficiency
"-1,980000","0,371072"
"-1,920000","0,479767"
"-1,900000","0,474569"
"-1,880000","0,595287"
"-1,780000","0,589183"


`df` has a signature

| speed | efficiency |
|-------|------------|

Let's take a look at `StatSmooth` output DataFrame:

In [4]:
df.statSmooth("speed", "efficiency").head(5)

Stat,,,,
x,y,yMin,yMax,se
"-1,980000","0,209863","-0,019837","0,439564","0,115015"
"-1,940000","0,255322","0,029081","0,481562","0,113282"
"-1,900000","0,301220","0,078421","0,524019","0,111559"
"-1,860000","0,347662","0,128286","0,567038","0,109845"
"-1,820000","0,394639","0,178665","0,610612","0,108142"


It has the following signature:

<table>
  <thead>
    <tr>
      <th alignt="left" colspan="5">Stat</th>
    </tr>
  </thead>
  <thead>
    <tr>
      <th>x</th>
      <th>y</th>
      <th>yMin</th>
      <th>yMax</th>
      <th>se</th>
    </tr>
  </thead>
</table>

As you can see, we got a `DataFrame` with one `ColumnGroup` called `Stat` which contains several columns with statics.
For `statSmooth`, each row corresponds to one of the line points.
`Stat.x` is the column with this point `x` coordinate.
`Stat.y` is points `y` coordinate; `Stat.yMin` — lower point of confidence level.
`Stat.yMax` — upper point of confidence level.
`Stat.se` — standard error.

`DataFrame` with "smooth" statistics is called `StatSmoothFrame`

### `statSmooth` transform

`statSmooth(statSmoothArgs) { /*new plotting context*/ }` modifies a plotting context —
instead of original data (no matter was it empty or not) new `StatSmooth` dataset
(calculated on given arguments. Inputs can be provided as `Iterable` or as dataset column reference —
by name as a `String`, as a `ColumnReference` or as a `DataColumn`) is used inside a new context
(original dataset and primary context are not affected —
you can add layers using initial dataset outside the `statSmooth` context).
Since the old dataset is irrelevant, we cannot use references for its columns.
But we can refer to the new ones.
They are all contained in the `Stat` group and can be called inside the new context:

In [5]:
plot {
    statSmooth(newXs, newYs) {
        // new `StatSmooth` dataset here
        area {
            // use `Stat.*` columns for mappings
            x(Stat.x)
            y(Stat.y)
        }
    }
    points {
        x(newXs)
        y(newYs)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Vzu0s1"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-1.98,-1.94,-1.9,-1.8599999999999999,-1.82,-1.78,-1.74,-1.7,-1.66,-1.62,-1.58,-1.54,-1.5,-1.46,-1.42,-1.38,-1.3399999999999999,-1.2999999999999998,-1.26,-1.22,-1.18,-1.1400000000000001,-1.1,-1.06,-1.02,-0.98,-0.94,-0.8999999999999999,-0.8599999999999999,-0.8200000000000001,-0.78,-0.74,-0.7,-0.6599999999999999,-0.6199999999999999,-0.5799999999999998,-0.54,-0.5,-0.45999999999999996,-0.41999999999999993,-0.3799999999999999,-0.33999999999999986,-0.30000000000000004,-0.26,-0.21999999999999997,-0.17999999999999994,-0.1399999999999999,-0.09999999999999987,-0.06000000000000005,-0.020000000000000018,0.020000000000000018,0.06000000000000005,0.10000000000000009,0.14000000000000012,0.18000000000000016,0.2200000000000002,0.26000000000000023,0.30000000000000027,0.33999999999999986,0.3799999999999999,0.41999999999999993,0.45999999999999996,0.5,0.54,0.5800000000000001,0.6200000000000001,0.6600000000000001,0.7000000000000002,0.7400000000000002,0.7800000000000002,0.8200000000000003,0.8599999999999999,0.8999999999999999,0.94,0.98,1.02,1.06,1.1,1.1400000000000001,1.1800000000000002,1.2200000000000002,1.2600000000000002,1.3000000000000003,1.3400000000000003,1.38,1.42,1.46,1.5,1.54,1.58,1.62,1.6600000000000001,1.7000000000000002,1.7400000000000002,1.7800000000000002,1.8200000000000003,1.8599999999999999,1.9,1.94,1.98],
"y":[0.20986322268519375,0.2553217335019903,0.3012199249481995,0.3476622596869782,0.39463875130392545,0.44215806220343756,0.4902265534468313,0.5388506585449422,0.5880363858297991,0.6377888013919245,0.6881164880755347,0.7390318102403889,0.7905473516347274,0.8426771703933615,0.8954414183127294,0.9488651314061353,1.0029735556539314,1.0577900687340296,1.1133405177356714,1.1696758130760612,1.2268379242857228,1.2841689137944214,1.340541253332233,1.3994630968405977,1.4647219589975458,1.5394642849015048,1.614634171714075,1.7050218346683779,1.8037238778386082,1.89328228657552,1.9826193433090973,2.0865655232730043,2.2151772506415486,2.359574022213319,2.5061411029429754,2.641263757785177,2.751327251694584,2.8227168496258557,2.841817816533652,2.7950154173726327,2.722976041550435,2.7147160215070363,2.7373200125835577,2.788072901202473,2.8026065442710886,2.8162085296158126,2.818383282072805,2.812676759970186,2.8135775157357195,2.7927785760042845,2.721972967410759,2.8596556119922543,2.843204192632879,2.809495352610704,2.7875329978752963,2.753646635443065,2.700658980616351,2.6390051308874627,2.5798859394960996,2.5578078826140347,2.6436339041830315,2.7040684752436377,2.619512996718372,2.512630554445369,2.416686666320832,2.324956719795411,2.230962538180425,2.1338894456705155,2.038052939590428,1.9432440644532067,1.8534829686551613,1.7649512782945591,1.6780987024106107,1.5949561908299894,1.5174720563581365,1.4456939603121721,1.3773265989245267,1.3137057824393266,1.2571588587703915,1.202510632302503,1.1487273814819845,1.0954033270230343,1.0423320534405864,0.9894882868773351,0.9368682434748683,0.884476734612085,0.8323493116213436,0.780518694028428,0.7290209341783249,0.6778887673170129,0.6271514577372164,0.5768255593801548,0.5269244485991712,0.47746150174760843,0.42844684924634824,0.37987763778642736,0.3317477681264217,0.2840578560596252,0.2368612340945011,0.18995876420454305]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area"

In [6]:
df.plot {
    statSmooth(speed, efficiency, method = SmoothMethod.Polynomial(2), smootherPointCount = 250) {
        ribbon {
            x(Stat.x)
            yMin(Stat.yMin)
            yMax(Stat.yMax)
        }
    }
    // Dataset is not changed here
    points {
        x(speed)
        y(efficiency)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="4VoR8Y"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"efficiency":[0.37107204228764407,0.4797671902415327,0.4745689060220055,0.5952866962907285,0.5891828031978252,0.42685758977309807,0.5869268128429572,0.40257260614904744,0.694852813784601,0.6418889518385589,0.9242265647836819,0.7439440664967731,0.878041250549323,1.1776413564509363,0.9318661477706146,1.0102218021885303,1.321993876565358,1.2122018296039632,1.5414389919459428,1.6022624375353698,1.6224029092777503,1.7875846286028219,2.819225383273898,3.2549891125150374,3.1136966524798786,3.197922638635543,3.2441073117388415,3.60090145896446,3.6402275884093553,3.839976876551691,3.7549379874541327,3.881375968592746,4.063611246888035,3.889509983463439,3.9361544504981225,3.837258466162532,4.011692198449596,3.1892648605850544,3.120237122197381,2.8829015435212977,2.6958368188468165,2.2983276703516826,1.980543630134041,1.9649350235037306,1.9477128254554883,1.837103737547647,1.8571642418362002,1.829633821126999,1.4076272113191926,1.2058137677091718,1.2063746771416133,1.2198591257506763,1.2391854713542136,1.0188484158407425,0.9900012854318744,1.0706559569580825,1.017231010182261,0.9840077762990119,0.8123358408401067,0.8106177439120912,0.6219431715448637,0.8082725654188169,0.8381470583141075,0.5911054906002517,0.4163787438080268,0.5596601801549382,0.3160741553626981],
"speed":[-1.98,-1.92,-1.9,-1.88,-1.78,-1.74,-1.7,-1.64,-1.52,-1.48,-1.34,-1.3,-1.24,-1.22,-1.18,-1.16,-1.0,-0.96,-0.94,-0.86,-0.84,-0.74,-0.42,-0.4,-0.36,-0.34,-0.32,-0.24,-0.2,-0.16,-0.14,0.02,0.04,0.06,0.08,0.14,0.2,0.34,0.38,0.46,0.5,0.64,0.66,0.72,0.74,0.76,0.8,0.82,0.96,1.04,1.06,1.08,1.1,1.14,1.16,1.2,1.24,1.26,1.38,1.42,1.46,1.48,1.58,1.74,1.88,1.9,1.98]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"yMin",
"ymax":"yMax"
},
"stat":"identity",
"data":{
"yMin":[-0.6938531877541734,-0.6390553495658592,-0.5846953702306585,-0.5307733201589218,-0.47728927133656457,-0.424243297364258,-0.37163547349759496,-0.3194658766882236,-0.267734585625987,-0.216441680782067,-0.1655872444531601,-0.11517136080669182,-0.06519411592709062,-0.015655597863126924,0.03344410332366543,0.08210489550948002,0.13032668445762513,0.17810937376680258,0.22545286481827603,0.2723570567219127,0.31882184626111665,0.36484712783662354,0.4104327934091893,0.4555787324411596,0.50028483183693,0.5445509758823144,0.5883770461828272,0.6317629216009109,0.6747084781921122,0.7172135891402527,0.759278124691617,0.8009019520881906,0.8420849355000045,0.8828269359566221,0.9231278112778298,0.9629874160035965,1.0024056013233649,1.0413822150047642,1.079917101321822,1.1180101009827765,1.1556610510575973,1.192869784905327,1.2296361321013873,1.2659599183649641,1.3018409654866585,1.3372790912565462,1.3722741093928343,1.4068258294713278,1.4409340568558895,1.4745985926301552,1.5078192335307297,1.5405957718821386,1.5729279955338278,1.6048156877994977,1.636258627399121,1.6672565884039803,1.6978093401850898,1.727916647365419,1.757578269776302,1.7867939624185039,1.8155634754283931,1.8438865540497145,1.871762938611481,1.8991923645125193,1.926174562213245,1.952709257235242,1.9787961701692762,2.004435016692369,2.029625507594591,2.0543673488162653,2.0786602414962654,2.1025038820321287,2.125897962152722,2.1488421690041863,2.171336185249924,2.1933796891853747,2.2149723548683347,2.2361138522655706,2.2568038474164718,2.277042002614453,2.296827976606826,2.316161424813799,2.335041999567248,2.3534693503698536,2.371443124175136,2.388962965688877,2.4060285176923246,2.4226394

### `smoothLine` layer

`smoothLine` layer is a shortcut for fast plotting a smoothed line:

In [7]:
val smoothLineLayerPlot = plot {
    smoothLine(newXs, newYs)
    layout.title = "`smoothLine()` layer"
}
smoothLineLayerPlot

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="pZTlxN"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"`smoothLine()` layer"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-1.98,-1.94,-1.9,-1.8599999999999999,-1.82,-1.78,-1.74,-1.7,-1.66,-1.62,-1.58,-1.54,-1.5,-1.46,-1.42,-1.38,-1.3399999999999999,-1.2999999999999998,-1.26,-1.22,-1.18,-1.1400000000000001,-1.1,-1.06,-1.02,-0.98,-0.94,-0.8999999999999999,-0.8599999999999999,-0.8200000000000001,-0.78,-0.74,-0.7,-0.6599999999999999,-0.6199999999999999,-0.5799999999999998,-0.54,-0.5,-0.45999999999999996,-0.41999999999999993,-0.3799999999999999,-0.33999999999999986,-0.30000000000000004,-0.26,-0.21999999999999997,-0.17999999999999994,-0.1399999999999999,-0.09999999999999987,-0.06000000000000005,-0.020000000000000018,0.020000000000000018,0.06000000000000005,0.10000000000000009,0.14000000000000012,0.18000000000000016,0.2200000000000002,0.26000000000000023,0.30000000000000027,0.33999999999999986,0.3799999999999999,0.41999999999999993,0.45999999999999996,0.5,0.54,0.5800000000000001,0.6200000000000001,0.6600000000000001,0.7000000000000002,0.7400000000000002,0.7800000000000002,0.8200000000000003,0.8599999999999999,0.8999999999999999,0.94,0.98,1.02,1.06,1.1,1.1400000000000001,1.1800000000000002,1.2200000000000002,1.2600000000000002,1.3000000000000003,1.3400000000000003,1.38,1.42,1.46,1.5,1.54,1.58,1.62,1.6600000000000001,1.7000000000000002,1.7400000000000002,1.7800000000000002,1.8200000000000003,1.8599999999999999,1.9,1.94,1.98],
"y":[0.20986322268519375,0.2553217335019903,0.3012199249481995,0.3476622596869782,0.39463875130392545,0.44215806220343756,0.4902265534468313,0.5388506585449422,0.5880363858297991,0.6377888013919245,0.6881164880755347,0.7390318102403889,0.7905473516347274,0.8426771703933615,0.8954414183127294,0.9488651314061353,1.0029735556539314,1.0577900687340296,1.1133405177356714,1.1696758130760612,1.2268379242857228,1.2841689137944214,1.340541253332233,1.3994630968405977,1.4647219589975458,1.5394642849015048,1.614634171714075,1.7050218346683779,1.8037238778386082,1.89328228657552,1.9826193433090973,2.0865655232730043,2.2151772506415486,2.359574022213319,2.5061411029429754,2.641263757785177,2.751327251694584,2.8227168496258557,2.841817816533652,2.7950154173726327,2.722976041550435,2.7147160215070363,2.7373200125835577,2.788072901202473,2.8026065442710886,2.8162085296158126,2.818383282072805,2.812676759970186,2.8135775157357195,2.7927785760042845,2.721972967410759,2.8596556119922543,2.843204192632879,2.809495352610704,2.7875329978752963,2.753646635443065,2.700658980616351,2.6390051308874627,2.5798859394960996,2.5578078826140347,2.6436339041830315,2.7040684752436377,2.619512996718372,2.512630554445369,2.416686666320832,2.324956719795411,2.230962538180425,2.1338894456705155,2.038052939590428,1.9432440644532067,1.8534829686551613,1.7649512782945591,1.6780987024106107,1.5949561908299894,1.5174720563581365,1.4456939603121721,1.3773265989245267,1.3137057824393266,1.2571588587703915,1.202510632302503,1.1487273814819845,1.0954033270230343,1.0423320534405864,0.9894882868773351,0.9368682434748683,0.884476734612085,0.8323493116213436,0.780518694028428,0.7290209341783249,0.6778887673170129,0.6271514577372164,0.5768255593801548,0.5269244485991712,0.47746150174760843,0.42844684924634824,0.37987763778642736,0.3317477681264217,0.2840578560596252,0.2368612340945011,0.18995876420454305]
},
"sampli

In [8]:
// Compare it with `statSmooth` + usual `line`
val statSmoothAndLinePlot = plot {
    statSmooth(newXs, newYs) {
        line {
            x(Stat.x)
            y(Stat.y)
        }
    }
    layout.title = "`statSmooth()` + non-statistical `line` layer"
}
plotGrid(listOf(smoothLineLayerPlot, statSmoothAndLinePlot))

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="qyDf4M"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"layout":{
"name":"grid",
"ncol":2,
"nrow":1,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"`smoothLine()` layer"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-1.98,-1.94,-1.9,-1.8599999999999999,-1.82,-1.78,-1.74,-1.7,-1.66,-1.62,-1.58,-1.54,-1.5,-1.46,-1.42,-1.38,-1.3399999999999999,-1.2999999999999998,-1.26,-1.22,-1.18,-1.1400000000000001,-1.1,-1.06,-1.02,-0.98,-0.94,-0.8999999999999999,-0.8599999999999999,-0.8200000000000001,-0.78,-0.74,-0.7,-0.6599999999999999,-0.6199999999999999,-0.5799999999999998,-0.54,-0.5,-0.45999999999999996,-0.41999999999999993,-0.3799999999999999,-0.33999999999999986,-0.30000000000000004,-0.26,-0.21999999999999997,-0.17999999999999994,-0.1399999999999999,-0.09999999999999987,-0.06000000000000005,-0.020000000000000018,0.020000000000000018,0.06000000000000005,0.10000000000000009,0.14000000000000012,0.18000000000000016,0.2200000000000002,0.26000000000000023,0.30000000000000027,0.33999999999999986,0.3799999999999999,0.41999999999999993,0.45999999999999996,0.5,0.54,0.5800000000000001,0.6200000000000001,0.6600000000000001,0.7000000000000002,0.7400000000000002,0.7800000000000002,0.8200000000000003,0.8599999999999999,0.8999999999999999,0.94,0.98,1.02,1.06,1.1,1.1400000000000001,1.1800000000000002,1.2200000000000002,1.2600000000000002,1.3000000000000003,1.3400000000000003,1.38,1.42,1.46,1.5,1.54,1.58,1.62,1.6600000000000001,1.7000000000000002,1.7400000000000002,1.7800000000000002,1.8200000000000003,1.8599999999999999,1.9,1.94,1.98],
"y":[0.20986322268519375,0.2553217335019903,0.3012199249481995,0.3476622596869782,0.39463875130392545,0.44215806220343756,0.4902265534468313,0.5388506585449422,0.5880363858297991,0.6377888013919245,0.6881164880755347,0.7390318102403889,0.7905473516347274,0.8426771703933615,0.8954414183127294,0.9488651314061353,1.0029735556539314,1.0577900687340296,1.1133405177356714,1.1696758130760612,1.2268379242857228,1.2841689137944214,1.340541253332233,1.3994630968405977,1.4647219589975458,1.5394642849015048,1.614634171714075,1.7050218346683779,1.8037238778386082,1.89328228657552,1.9826193433090973,2.0865655232730043,2.2151772506415486,2.359574022213319,2.5061411029429754,2.641263757785177,2.751327251694584,2.8227168496258557,2.841817816533652,2.7950154173726327,2.722976041550435,2.7147160215070363,2.7373200125835577,2.788072901202473,2.8026065442710886,2.8162085296158126,2.818383282072805,2.812676759970186,2.8135775157357195,2.7927785760042845,2.721972967410759,2.8596556119922543,2.843204192632879,2.809495352610704,2.7875329978752963,2.753646635443065,2.700658980616351,2.6390051308874627,2.5798859394960996,2.5578078826140347,2.6436339041830315,2.7040684752436377,2.619512996718372,2.512630554445369,2.416686666320832,2.324956719795411,2.230962538180425,2.1338894456705155,2.038052939590428,1.9432440644532067,1.8534829686551613,1.7649512782945591,1.6780987024106107,1.5949561908299894,1.5174720563581365,1.4456939603121721,1.3773265989245267,1.3137057824393266,1.2571588587703915,1.202510632302503,1.1487273814819845,1.0954033270230343,1.0423320534405864,0.9894882868773351,0.9368682434748683,0.884476734612085,0.8323493116213436,0.780518694028428,0.7290209341783249,0.6778887673170129,0.6271514577372164,0.5768255593801548,0.5269244485991712,0.47746150174760843,0.42844684924634824,0.37987763778642736,

`smoothLine` uses `statSmooth` and `line` and performs coordinate mappings under the hood.
And we can customize `smoothLine` layer: `smoothLine()` optionally opens a new context,
where we can configure the line (as in the usual context opened by `line { ... }`) —
even change coordinate mappings from default ones.
`StatSmooth` dataset of `smoothLine` is also can be accessed here.

In [9]:
df.plot {
    smoothLine(speed, efficiency, SmoothMethod.LOESS(span = 0.1), 120) {
        // change a column mapped on `y` to `Stat.scaled`
        y(Stat.yMax)
        color = Color.RED
        width = 4.0
    }
    points {
        x(speed)
        y(efficiency)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="gAyb0d"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"efficiency":[0.37107204228764407,0.4797671902415327,0.4745689060220055,0.5952866962907285,0.5891828031978252,0.42685758977309807,0.5869268128429572,0.40257260614904744,0.694852813784601,0.6418889518385589,0.9242265647836819,0.7439440664967731,0.878041250549323,1.1776413564509363,0.9318661477706146,1.0102218021885303,1.321993876565358,1.2122018296039632,1.5414389919459428,1.6022624375353698,1.6224029092777503,1.7875846286028219,2.819225383273898,3.2549891125150374,3.1136966524798786,3.197922638635543,3.2441073117388415,3.60090145896446,3.6402275884093553,3.839976876551691,3.7549379874541327,3.881375968592746,4.063611246888035,3.889509983463439,3.9361544504981225,3.837258466162532,4.011692198449596,3.1892648605850544,3.120237122197381,2.8829015435212977,2.6958368188468165,2.2983276703516826,1.980543630134041,1.9649350235037306,1.9477128254554883,1.837103737547647,1.8571642418362002,1.829633821126999,1.4076272113191926,1.2058137677091718,1.2063746771416133,1.2198591257506763,1.2391854713542136,1.0188484158407425,0.9900012854318744,1.0706559569580825,1.017231010182261,0.9840077762990119,0.8123358408401067,0.8106177439120912,0.6219431715448637,0.8082725654188169,0.8381470583141075,0.5911054906002517,0.4163787438080268,0.5596601801549382,0.3160741553626981],
"speed":[-1.98,-1.92,-1.9,-1.88,-1.78,-1.74,-1.7,-1.64,-1.52,-1.48,-1.34,-1.3,-1.24,-1.22,-1.18,-1.16,-1.0,-0.96,-0.94,-0.86,-0.84,-0.74,-0.42,-0.4,-0.36,-0.34,-0.32,-0.24,-0.2,-0.16,-0.14,0.02,0.04,0.06,0.08,0.14,0.2,0.34,0.38,0.46,0.5,0.64,0.66,0.72,0.74,0.76,0.8,0.82,0.96,1.04,1.06,1.08,1.1,1.14,1.16,1.2,1.24,1.26,1.38,1.42,1.46,1.48,1.58,1.74,1.88,1.9,1.98]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"speed",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"efficiency",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"yMax"
},
"stat":"identity",
"data":{
"yMax":[0.4205277433192251,0.4747337767045443,0.5257020407431814,0.5693444471293232,0.5943114331775364,0.603203780238874,0.6045155867508929,0.601724221040826,0.5849540526677364,0.5917061671718091,0.6280295575366888,0.6579807660355008,0.6745234027062653,0.6848178303029616,0.696098764963175,0.7132239889449363,0.7352300861897714,0.7596417319802492,0.7837983701708576,0.8050394512085308,0.8220646854265352,0.8444362426388929,0.8819691417712325,0.9363138135176516,0.9898757278013905,1.0394860768278857,1.0856889085273216,1.1376486617931847,1.205398371085762,1.2989710838635153,1.423280527861529,1.4932780315937264,1.5247521249746234,1.5891472210785227,1.650396612873677,1.6962524884541665,1.7404047422928075,1.792454099538559,1.861067650529096,1.9470332798190417,2.046939119333968,2.157334103690905,2.274767172945953,2.395787270609661,2.516943341397626,2.6347843287169534,2.7458591718959355,2.8467817501917025,2.977016487506156,3.1465610004240365,3.3114630820103965,3.4722418187421598,3.5955427013803765,3.6630124909768678,3.7741661686176524,3.847560311202748,3.8107038619894538,3.7868541392634913,3.78784328880261,3.8235975201044208,3.904042996310163,3.960493241612471,3.956927667126842,3.904621316256443,3.8904818092116353,3.9700505295414406,3.9776630995733995,3.846728880799399,3.635437961809655,3.4076665313811705,3.2267488752621163,3.107630873748496,2.9968590081172732,2.8872726049455597,2.7821423374954453,2.6806156204731844,2.5809299232322043,2.483120111810828,2.3873107747803917,2.2931979821274666,2.1680107119908,2.0296928

## Grouped `statSmoth`

`statSmooth` can be applied for grouped data —
statistics will be calculated on each group independently but with equal categories.
This application returns a new `GroupBy`
dataset with the same keys as the old one but with `StatSmooth` groups instead of old ones.

In [10]:
// Generate two lines
val fA = { x: Double -> 0.02 * x * x * x - 0.2 * x * x + 0.1 * x + 2.1 }
val fB = { x: Double -> -0.1 * x * x * x + 0.5 * x * x - 0.8 }
val xRange = (-500..500).map { it / 100.0 }
val noisesA = NormalDistribution(0.0, 0.05).sample(xRange.size).toList()
val noisesB = NormalDistribution(0.0, 0.2).sample(xRange.size).toList()
val valuesA = xRange.zip(noisesA).map { fA(it.first) + it.second }
val valuesB = xRange.zip(noisesB).map { fB(it.first) + it.second }

val (xsA, ysA) = xRange.zip(valuesA).shuffled(Random(17)).take(xRange.size * 1 / 3).sortedBy { it.first }
    .unzip()
val (xsB, ysB) = xRange.zip(valuesB).shuffled(Random(17)).take(xRange.size * 1 / 3).sortedBy { it.first }
    .unzip()

In [11]:
// Gather them into `DataFrame` with "A" and "B" keys in "category" column
val valuesDF = dataFrameOf(
    "time" to xsA + xsB,
    "value" to ysA + ysB,
    "category" to List(xsA.size) { "A" } + List(xsB.size) { "B" }
)
valuesDF.head(5)

time,value,category
"-4,960000","-5,780055",A
"-4,890000","-5,427302",A
"-4,870000","-5,493057",A
"-4,840000","-5,368819",A
"-4,830000","-5,293401",A


It has the following signature:

| time | value | category |
|------|-------|----------|

In [12]:
// Group it by "category"
val groupedDF = valuesDF.groupBy { category }
groupedDF

<iframe onload="o_resize_iframe_out_6()" style="width:100%;" class="result_container" id="iframe_out_6" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
 white-space: nowrap;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-771751926"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.lengt

Now we have a `GroupBy` with a signature

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: DataFrame[time|value|category]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

In [13]:
groupedDF.statSmooth { x(time); y(value) }

<iframe onload="o_resize_iframe_out_8()" style="width:100%;" class="result_container" id="iframe_out_8" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
 white-space: nowrap;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-771751918"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.lengt

After `statSmooth` applying it's still a `GroupBy` but with different signature of `group` —
all groups have the same signature as usual `DataFrame` after `statSmooth` applying (i.e. `StatSmoothFrame`):

<table>
  <thead>
    <tr>
      <th>key: [drv]</th>
      <th>group: StaSmoothFrame</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>"A"</td>
      <td>"A"-Group</td>
    </tr>
    <tr>
      <td>"B"</td>
      <td>"B"-Group</td>
    </tr>
  </tbody>
</table>

As you can see, we did indeed do a `statSmooth` transformation within groups, the grouping keys did not change.

The plotting process doesn't change much — we do everything the same.

In [14]:
groupedDF.plot {
    statSmooth(time, value) {
        line {
            x(Stat.x)
            y(Stat.y)
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="x15DSI"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-

As you can see, there are two lines because we have two groups of data.
To distinguish them, we need to add mapping to the color from the key.
This is convenient — the key is available in the context

In [15]:
groupedDF.plot {
    statSmooth(time, value, method = SmoothMethod.Polynomial(3)) {
        line {
            x(Stat.x)
            y(Stat.y)
            color(key.category)
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="JMiofy"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,

The `smoothLine()` layer also works.
Moreover, if we have exactly one grouping key, a mapping from it to `color` will be created by default.

In [16]:
groupedDF.plot {
    smoothLine(time, value)
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="kqow8r"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"time",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"value",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-

We can customize it like we used to. From the differences — access to `key` columns:

In [17]:
groupedDF.plot {
    smoothLine(time, value) {
        color = Color.GREEN
        type(key.category)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="gqlIqp"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"time",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"value",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"linetype"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"linetype":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272

### Inside `groupBy{}` plot context

We can apply `groupBy` modification to the initial dataset and build a histogram with grouped data the same way:

In [18]:
valuesDF.plot {
    groupBy(category) {
        smoothLine(time, value)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="3cWpXY"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"time",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"value",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-